In [30]:
import numpy as np
import pandas as pd
from datetime import datetime
import os,glob

In [15]:
main_datafile_path = 'data/'
temp_files_path = 'data/temp_files/'

now = datetime.now()
f_date = now.strftime('%Y%m%d')


In [16]:
def stn_name_modification(f_date):
    df_st = pd.read_csv(f'{main_datafile_path}metro_ridership_by_line_stn_time.csv',encoding='euc-kr')
    df_st['지하철역'] = df_st['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
    for i in df_st.index:
        if df_st.loc[i, '지하철역'][-1] == '역':
            df_st.loc[i, '지하철역'] = df_st.loc[i, '지하철역'][:-1]
            
    df_st.to_csv(f'data/{f_date}_info.csv',index=False,encoding='utf-8')
    return None

In [24]:
def line_sep_preproc_main(f_date):
    stn_name_modification(f_date)
    df = pd.read_csv(f'data/{f_date}_info.csv')
    lines = df.호선명.unique().tolist()
    df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
    for line, frame in df_dict.items():
        # frame = df[df['호선명']==line].copy()
        frame['새벽 승차인원'] = frame.loc[:,['04시-05시 승차인원','05시-06시 승차인원']].sum(axis=1)
        frame['새벽 하차인원'] = frame.loc[:,['04시-05시 하차인원','05시-06시 하차인원']].sum(axis=1)

        frame['출근시간 승차인원'] = frame.loc[:,['06시-07시 승차인원','07시-08시 승차인원','08시-09시 승차인원']].sum(axis=1)
        frame['09-16시 승차인원'] = frame.loc[:,['09시-10시 승차인원','10시-11시 승차인원','11시-12시 승차인원',
                                                '12시-13시 승차인원','13시-14시 승차인원','14시-15시 승차인원','16시-17시 승차인원']].sum(axis=1)
        frame['퇴근시간 승차인원'] = frame.loc[:,['17시-18시 승차인원','18시-19시 승차인원','19시-20시 승차인원']].sum(axis=1)
        frame['야간 승차인원'] = frame.loc[:,['20시-21시 승차인원','21시-22시 승차인원','22시-23시 승차인원',
                                            '23시-24시 승차인원','00시-01시 승차인원','01시-02시 승차인원']].sum(axis=1)
        frame['출근시간 하차인원'] = frame.loc[:,['06시-07시 하차인원','07시-08시 하차인원','08시-09시 하차인원']].sum(axis=1)
        frame['퇴근시간 하차인원'] = frame.loc[:,['17시-18시 하차인원','18시-19시 하차인원','19시-20시 하차인원']].sum(axis=1)
        frame['09-16시 하차인원'] = frame.loc[:,['09시-10시 하차인원','10시-11시 하차인원','11시-12시 하차인원',
                                                '12시-13시 하차인원','13시-14시 하차인원','14시-15시 하차인원','16시-17시 하차인원']].sum(axis=1)
        frame['야간 하차인원'] = frame.loc[:,['20시-21시 하차인원','21시-22시 하차인원','22시-23시 하차인원',
                                            '23시-24시 하차인원','00시-01시 하차인원','01시-02시 하차인원']].sum(axis=1)
        frame['총 승차인원'] = frame.loc[:,['새벽 승차인원','출근시간 승차인원','09-16시 승차인원','야간 승차인원']].sum(axis=1)
        frame['총 하차인원'] = frame.loc[:,['새벽 하차인원','출근시간 하차인원','09-16시 하차인원','야간 하차인원']].sum(axis=1)
        frame = frame[['사용월', '호선명', '지하철역', '출근시간 승차인원', '출근시간 하차인원', 
                        '09-16시 승차인원', '09-16시 하차인원', '퇴근시간 승차인원', '퇴근시간 하차인원',
                        '새벽 승차인원','새벽 하차인원','야간 승차인원', '야간 하차인원','총 승차인원','총 하차인원']]
        
        frame.loc[(frame['호선명'] == '2호선') & (frame['지하철역'] == '신천'), '지하철역'] = '잠실새내'

        frame.to_csv(f'{temp_files_path}{line}.csv',index=False,encoding='utf-8')
    return None

In [31]:
def rm_temp_files():
    temp_path = os.path.join('data', 'temp_files', '*.csv')
    for file in glob.glob(temp_path):
        os.remove(file)
    return None

In [32]:
def preproc_main(f_date):
    line_sep_preproc_main(f_date)
    
    path = 'data/temp_files/'
    line_info = [
        ([f'{path}1호선.csv', f'{path}경부선.csv', f'{path}경원선.csv', f'{path}경인선.csv', f'{path}장항선.csv'], '1호선'),
        ([f'{path}2호선.csv'], '2호선'),
        ([f'{path}3호선.csv', f'{path}일산선.csv'], '3호선'),
        ([f'{path}4호선.csv', f'{path}과천선.csv', f'{path}안산선.csv'], '4호선'),
        ([f'{path}5호선.csv'], '5호선'),
        ([f'{path}6호선.csv'], '6호선'),
        ([f'{path}7호선.csv'], '7호선'),
        ([f'{path}8호선.csv'], '8호선'),
        ([f'{path}9호선.csv', f'{path}9호선2단계.csv', f'{path}9호선2~3단계.csv'], '9호선'),
        ([f'{path}수인선.csv', f'{path}분당선.csv'], '수인분당선'),
        ([f'{path}경의선.csv', f'{path}중앙선.csv'], '경의중앙선'),
        ([f'{path}공항철도 1호선.csv'], '공항철도')
    ]
    line_list = []
    for df_list, line_name in line_info:
        df_copies = []
        for file in df_list:
            df = pd.read_csv(file)
            df_copies.append(df.copy())
        result = pd.concat(df_copies, axis=0)
        result = result.reset_index(drop=True)
        result.호선명 = line_name
        cols = list(result.columns)[:3]
        target = list(result.columns)[3:]
        res = result.groupby(cols)[target].agg('sum').reset_index()
        line_list.append(res)
    final = pd.concat(line_list,axis=0)
    rm_temp_files()
    final.to_csv('data/merged_lines.csv',index=False)
    return None

In [33]:
preproc_main(f_date)